In [2]:
from os import environ
from iexfinance.stocks import Stock
from dotenv import load_dotenv
from datetime import datetime
from iexfinance.stocks import Stock
import pyodbc
import sqlalchemy
import pandas
import urllib
from sqlalchemy import create_engine

# splits a list into chunks of the given size
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [2]:
# load the environment variables and verity we have a token
load_dotenv()
iex_token = [environ[key] for key in environ if key == 'IEX_TOKEN']
print(iex_token[0][0:10])

pk_f401aad


In [6]:
# connect to the database
drivers = [x for x in pyodbc.drivers() if x.endswith(' for SQL Server')]
print(drivers)
(svr, db, uid, pwd) = environ['MSSQL_SERVER'], environ['MSSQL_DB'], environ['MSSQL_UID'], environ['MSSQL_PWD']
conn = pyodbc.connect(f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={svr};DATABASE={db};UID={uid};PWD={pwd}')

['ODBC Driver 17 for SQL Server']


In [7]:
# get the symbols
sql = "Exec usp_Equity_Read"
symbols = pandas.read_sql(sql,conn)
symbols['Symbol'] = symbols['Symbol'].str.upper()
symbol_list = symbols['Symbol'].tolist()
symbols.head()

,EquityID,Symbol,LastProcessDate
0,3,AAPL,2021-01-04 23:40:16.913
1,13,ADBE,2021-01-06 12:10:13.453
2,32,AMD,2021-01-03 14:30:11.947
3,34,AMGN,2021-01-06 15:00:10.470
4,125,CGNX,2021-01-03 12:20:10.353


In [8]:
# get the trade date id
today = datetime.today().strftime('%Y-%m-%d')
# today = '2020-12-16'
sql = f"SELECT TradedateID FROM dbo.TradeDate WHERE TradeDate = '{today}'"
cursor = conn.cursor()
cursor.execute(sql)
tradeDateID = cursor.fetchone()[0]

In [15]:
# pull the data from iex and add it to the data frame
# iex can only get 100 records at a time
chunks = chunker(symbol_list, 100)
data = symbols[['EquityID', 'Symbol']]
quotes = []
for chunk in chunks:
    batch = Stock(chunk, output_format="pandas")
    quote = batch.get_quote()
    quote = quote[['symbol', 'iexOpen', 'iexClose', 'high', 'low', 'iexVolume']]
    quotes.append(quote)
quotes = pandas.concat(quotes)
data = data.merge(quotes, left_on='Symbol', right_on='symbol', how='outer')
data.head()

,EquityID,Symbol,symbol,iexOpen,iexClose,high,low,iexVolume
0,3,AAPL,AAPL,126.88,126.70,None,None,2202330
1,13,ADBE,ADBE,466.22,466.14,None,None,240255
2,32,AMD,AMD,90.32,90.32,None,None,696645
3,34,AMGN,AMGN,233.37,233.37,None,None,231758
4,125,CGNX,CGNX,83.68,83.68,None,None,101520


In [16]:
# make the columns match the database table
data.rename(columns = {
    'iexOpen': 'Open', 
    'iexClose': 'Close', 
    'high': 'High',
    'low': 'Low',
    'iexVolume': 'Volume'
}, inplace = True) 
data.drop(columns=['symbol'], inplace = True)
data['TradeDate'] = today
data['TradeDateID'] = tradeDateID
data['LastModifiedDate'] = datetime.now()
data.head()

,EquityID,Symbol,Open,Close,High,Low,Volume,TradeDate,TradeDateID,LastModifiedDate
0,3,AAPL,126.88,126.70,None,None,2202330,2021-01-06,13528,2021-01-06 16:29:14.499150
1,13,ADBE,466.22,466.14,None,None,240255,2021-01-06,13528,2021-01-06 16:29:14.499150
2,32,AMD,90.32,90.32,None,None,696645,2021-01-06,13528,2021-01-06 16:29:14.499150
3,34,AMGN,233.37,233.37,None,None,231758,2021-01-06,13528,2021-01-06 16:29:14.499150
4,125,CGNX,83.68,83.68,None,None,101520,2021-01-06,13528,2021-01-06 16:29:14.499150


In [17]:
# write to the database
(svr, db, uid, pwd) = environ['MSSQL_SERVER'], environ['MSSQL_DB'], environ['MSSQL_UID'], environ['MSSQL_PWD']
conn = pyodbc.connect(f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={svr};DATABASE={db};UID={uid};PWD={pwd}')

quoted = urllib.parse.quote_plus(f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={svr};DATABASE={db};UID={uid};PWD={pwd}')
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={quoted}')

data.to_sql('Stage_IEXData', schema='dbo', con=engine, if_exists='append', index=False)

In [3]:
appl = Stock(['AAPL'], output_format="pandas")
quote = appl.get_quote()
#quote.head()
quote.to_csv('quote.csv')

In [3]:
def is_market_open():
    appl = Stock(['AAPL'], output_format="json").get_quote()
    print(appl.get('isUSMarketOpen'))


is_market_open()

IEXQueryError: An error occurred while making the query (403): The API key provided is not valid.

In [4]:
appl = Stock(['AAPL']).get_quote()
appl.to_csv('appl2.csv')